In [ ]:
import torch
import numpy as np
import pandas as pd
import torchvision
import torch.nn as nn
from PIL import Image
from tqdm.notebook import tqdm
import torchvision.models as models
from torchvision import transforms, models

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
patch_size = {PATCH_SIZE} 
patch_dir = 'PATH-TO-PATCH-DIRECTORY' 

**PREPARING CLINICAL DATA**

In [ ]:
# reading patch data

patch_info = pd.read_csv("PATH-TO-PATCH-INFO-DATA-CSV")
patch_info.head(3) 

In [ ]:
# obtaining patch paths

def obtain_patch_path(row): 
    
    wsi = str(row['WSI No.'])
    x = str(row['X Pixel'])
    y = str(row['Y Pixel'])
    
    return f'{patch_dir}/{wsi}_{x}_{y}.jpg'

df['Patch Path'] = df.apply(obtain_patch_path, axis=1) 
df.head(3) 

In [ ]:
# populating lists

wsi_nos = df['WSI No.'].to_list() 
x_pixels = df['X Pixel'].to_list() 
y_pixels = df['Y Pixel'].to_list() 
annotation_ids = df['Annotation Id'].to_list() 
patch_paths = df['Patch Path'].to_list() 

**FEATURE EXTRACTOR** 

In [ ]:
# feature extractor for patches 

class Pipeline(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.model = models.{'MODEL-NAME'}(weights=True) 
        
        # Freeze model weights
        for param in self.model.parameters():
            param.requires_grad = False 
        
    def forward(self,X):
        X = self.model(X)
        return X        
            
model = Pipeline(1) 
model = model.to(device) 

In [ ]:
# transform function for patches 

transform = torchvision.transforms.Compose(
    [ 
        torchvision.transforms.Resize(224), 
        torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # normalization
    ] 
) 

In [ ]:
# function to obtain feature embedding for a given patch 

def get_feature_vector(img): 
    img = torch.from_numpy(img.astype(np.double)) 
    img = img.permute(2, 0, 1) 
    img = transform(img) 
    img = img.float() 
    img = torch.unsqueeze(img, dim=0) 
    img = img.to(device) 
    return model(img) 

**OBTAINING EMBEDDINGS** 

In [ ]:
# function to return feature vectors along with labels for patches 

def get_patch_embeddings(patch_paths, annotation_ids): 

    embeds_count = [] 
    feature_vectors = []
    count = 1
    
    for i in tqdm(range(len(patch_paths))): 

        if annotation_ids[i] != 0: 
        
            # Load image
            img = Image.open(patch_paths[i]).convert('RGB')  # Ensure RGB mode
    
            # Convert to numpy array
            img_np = np.array(img)
    
            # Obtain feature vector
            feature_vector = get_feature_vector(img_np)
            squeezed_arr = torch.squeeze(feature_vector).detach().cpu().numpy()
    
            feature_vectors.append(squeezed_arr) 
            embeds_count.append(count) 
            count += 1

        else: 
            embeds_count.append(-1) 

    return feature_vectors, embeds_count 

In [ ]:
# obtaining patch embeddings for all wsis 

patch_embeddings, embedding_nos = get_patch_embeddings(patch_paths, annotation_ids) 

In [ ]:
print(len(patch_embeddings), len(embedding_nos))

**SAVING PATCH EMBEDDINGS** 

In [ ]:
# storing patch embeddings in pth file 

torch.save(patch_embeddings,'PATH-FOR-PATCH-EMBEDDINGS(.pth)') 

**STORING UPDATED CSV** 

In [ ]:
df['Embedding No.'] = embedding_nos 

In [ ]:
df.to_csv('PATH-TO-PATCH-INFO-CSV', index=False) 